## 集合竞价画图工具

In [ ]:
# 这里给大家 展示一下怎么运用咱们supermind的 tick级别数据制作集合竞价 高频因子 的方法
#一般竞价多头总数>0的股票 证明 集合竞价主力有在运作，在该日博弈有较大概率 可以买到涨停的股票
import pandas as pd
import numpy as np
import datetime
from mindgo_api import get_tick
import matplotlib.pyplot as plt  

def JJ(ks,s1):
    cf=get_tick(ks,"{0} 09:20".format(s1),"{0} 09:25:00".format(s1),['total_bid','total_ask','volume','b1_v','b2_v','b3_v','a1_v','a2_v','a3_v','cjcs'])
    cf = cf.fillna(0)
    cf['多空线_1']=cf['total_bid']-cf['total_ask']

    cf['竞价线']=cf['b1_v']+cf['b2_v']-cf['a1_v']-cf['a2_v']
        # 设置画布大小
    plt.figure(figsize=(10, 5))
    cf['trade_date'] = pd.to_datetime(cf['trade_date'])  # 将日期字符串转换为时间序列类型

    cf['竞价成交量因子']=cf['竞价线'].values[-1]-cf['竞价线'].values[0]
    cf['竞价多头总数因子']=(cf['竞价线'].values >0).sum()


    df_new = cf.drop_duplicates(subset=['id_stock', '竞价成交量因子','竞价多头总数因子'], keep='last')
    data = df_new[['id_stock','竞价成交量因子','竞价多头总数因子']]
    data.columns = ['股票代码','竞价成交量因子','竞价多头总数因子']    
    # 准备用于图例显示的股票代码列表（这里确保id_stock列的数据是唯一的，符合图例一对一标识要求）
    stock_codes_for_legend = data['股票代码'].tolist()  

    # 绘制竞价线图形，以日期为横坐标，竞价线为纵坐标
    plt.plot(cf['trade_date'], cf['竞价线'])

    # 添加图例，用股票代码列表作为图例文本，使每条线对应相应的股票代码
    plt.legend(stock_codes_for_legend)  
    plt.plot(cf['trade_date'],cf['竞价线'])

    print(data)

## 沪深主板排除ST，股票市值20-500亿之间；2025-01-02涨停；2025-01-02情绪温度在95以上；2025-01-03竞价强度排名和股性评分和竞价金额与昨日最大一分钟成交额占比和竞价金额占昨日成交额；2025-01-03竞价金额与昨日最大一分钟成交额占比小于150%

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import time
import numpy as np
from tabulate import tabulate
from mindgo_api import *


pd.options.display.max_columns = None  # 显示所有列
pd.options.display.max_rows = None  # 显示所有行
  
part1 = "沪深主板排除ST，股票市值20-500亿之间；"
part2 = "涨停；"
part3 = "情绪温度在95以上；"
part4 = "竞价强度排名和股性评分和竞价金额与昨日最大一分钟成交额占比和竞价金额占昨日成交额；"
part5 = "竞价金额与昨日最大一分钟成交额占比小于150%"


# 获取当前日期和时间
end_date = datetime.now()

tdays_list = get_trade_days(end_date=end_date.strftime('%Y-%m-%d'),count=10)
# 假设要打印过去30天，你可以根据实际需求修改这个天数

for i in range(len(tdays_list) - 1, 0, -1):
    yes_current_date = tdays_list[i-1].strftime('%Y-%m-%d')
    current_date = tdays_list[i].strftime('%Y-%m-%d')
    new_query = "{}{}{}{}{}{}{}{}{}".format(part1, yes_current_date, part2,yes_current_date, part3, current_date, part4, current_date, part5)
    print(new_query)

    count = 0
    while count<100:
        print('开始循环:',count)
        time.sleep(1)
        count+=1
        result = query_iwencai(new_query,domain='股票',timeout=10)
        if len(result)>0:
            stock_code_list = result['股票代码'].tolist()
#             for code in stock_code_list:
#                 JJ(code,current_date)
            log.info(result)
            break

    print('循环结束')

## 情绪热点

In [44]:
import datetime
import pandas as pd
import time
import numpy as np
from tabulate import tabulate
from mindgo_api import *

def format_date_to_chinese(date_str):
    """
    将格式为%Y-%m-%d的日期字符串转换为中文日期格式（%Y年%m月%d日）
    """
    date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    return date_obj.strftime('%Y年%m月%d日')


pd.options.display.max_columns = None  # 显示所有列
pd.options.display.max_rows = None  # 显示所有行
  
query_1_part1 = "沪深主板排除ST，股票市值20-500亿之间；"
query_1_part2 = "成交额大于前一日成交额的0.9倍；"
query_1_part22 = "涨停；"
query_1_part3 = "竞价强度排名；"
query_1_part41 = "情绪温度相比前一日提升排名；"
# part41 = "情绪温度相比前一日提升提升了20 分以上；"
query_1_part42 = "情绪温度；"
query_1_part5 = "竞价金额与昨日最大一分钟成交额占比小于150%；"
query_1_part6 = "竞价评级是看多；"
query_1_part7 = "开盘价大于前一日收盘价的1.02倍；"



# 竞价是看多或者空：共交易【 172 】次，成功了【 125 】次，胜率【 0.7267441860465116 %】收益率【 0 】
# 竞价看多：共交易【 108 】次，成功了【 79 】次，胜率【 0.7314814814814815 %】收益率【 0 】
# 只看 95 以上：共交易【 83 】次，成功了【 60 】次，胜率【 0.7228915662650602 %】收益率【 0 】
# 沪深主板排除ST，股票市值20-500亿之间； 2024-12-26涨停； 2024-12-27竞价强度排名； 2024-12-26情绪温度环比提升排名第一； 2024-12-26情绪温度； 2024-12-27竞价金额与昨日最大一分钟成交额占比小于150%； 2024-12-27竞价评级是看多；:共交易【 134 】次，成功了【 103 】次，胜率【 0.7686567164179104 %】收益率【 0 】
# 沪深主板排除ST，股票市值20-500亿之间； 2024-10-25涨停； 2024-10-28竞价强度排名； 2024-10-25情绪温度环比提升排名第一； 2024-10-25情绪温度； 2024-10-28竞价金额与昨日最大一分钟成交额占比小于150%； 2024-10-28竞价评级是看多；:共交易【 142 】次，成功了【 110 】次，胜率【 0.7746478873239436 %】收益率【 0 】
#  只买最强。并且超级严格模式：共交易【 48 】次，成功了【 43 】次，胜率【 0.8958333333333334 %】收益率【 0 】
# 买第二强。并且超级严格模式：共交易【 48 】次，成功了【 39 】次，胜率【 0.8125 %】收益率【 0 】
# 只买最强回测 500 天：共交易【 476 】次，成功了【 287 】次，胜率【 0.6029411764705882 %】收益率【 0 】
#  排除一字板之后，只买最强的胜率回测 50 天：共交易【 45 】次，成功了【 32 】次，胜率【 0.7111111111111111 %】收益率【 0 】
# 排除一字板之后，只买2强的胜率回测 500 天 共交易【 840 】次，成功了【 398 】次，胜率【 0.4738095238095238 %】收益率【 0 】
#   只买最强：一字板共交易【 45 】次，成功了【 45 】次，胜率【 1.0 %】收益率【 0 】
#  只买最强：非一字板共交易【 48 】次，成功了【 24 】次，胜率【 0.5 %】收益率【 0 】
#  只买最强：
# 2024-10-28 :开盘未涨停：共交易【 42 】次，成功了【 26 】次，胜率【 0.6190476190476191 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 46 】次，成功了【 44 】次，胜率【 0.9565217391304348 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 40 】次，成功了【 40 】次，胜率【 1.0 %】收益率【 0 】
# 只买最强，大于 80：
# 2024-10-28 :开盘未涨停：共交易【 24 】次，成功了【 14 】次，胜率【 0.5833333333333334 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 39 】次，成功了【 33 】次，胜率【 0.8461538461538461 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 30 】次，成功了【 30 】次，胜率【 1.0 %】收益率【 0 】
# 只买最强，大于 80：
# 2024-10-28 :开盘未涨停：共交易【 24 】次，成功了【 14 】次，胜率【 0.5833333333333334 %】收益率【 0 】
# 2024-10-28 :开盘涨停非一字板：共交易【 30 】次，成功了【 20 】次，胜率【 0.6666666666666666 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 30 】次，成功了【 30 】次，胜率【 1.0 %】收益率【 0 】
# 只买最强，大于 60：
# 2024-10-28 :开盘未涨停：共交易【 38 】次，成功了【 26 】次，胜率【 0.6842105263157895 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 34 】次，成功了【 22 】次，胜率【 0.6470588235294118 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 38 】次，成功了【 38 】次，胜率【 1.0 %】收益率【 0 】
# 只买最强
# 2024-10-28 :开盘未涨停：共交易【 42 】次，成功了【 26 】次，胜率【 0.6190476190476191 %】收益率【 0 】
# 2024-10-28 :开盘涨停非一字：共交易【 41 】次，成功了【 31 】次，胜率【 0.7560975609756098 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 44 】次，成功了【 44 】次，胜率【 1.0 %】收益率【 0 】
# 只买最强，3 版以内
# 2024-10-28 :开盘未涨停：共交易【 42 】次，成功了【 26 】次，胜率【 0.6190476190476191 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 39 】次，成功了【 31 】次，胜率【 0.7948717948717948 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 42 】次，成功了【 42 】次，胜率【 1.0 %】收益率【 0 】
# 最近10个交易日涨停板小于3
# 2024-10-28 :开盘未涨停：共交易【 34 】次，成功了【 23 】次，胜率【 0.6764705882352942 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 27 】次，成功了【 20 】次，胜率【 0.7407407407407407 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 28 】次，成功了【 28 】次，胜率【 1.0 %】收益率【 0 】
# 最近10个交易日涨停板小于4
# 2024-10-28 :开盘未涨停：共交易【 36 】次，成功了【 23 】次，胜率【 0.6388888888888888 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 30 】次，成功了【 21 】次，胜率【 0.7 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 31 】次，成功了【 31 】次，胜率【 1.0 %】收益率【 0 】
# 最近 10 天涨幅大于 0，小于 30
# 2024-10-28 :开盘未涨停：共交易【 40 】次，成功了【 25 】次，胜率【 0.625 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 34 】次，成功了【 27 】次，胜率【 0.7941176470588235 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 40 】次，成功了【 40 】次，胜率【 1.0 %】收益率【 0 】
# 高开 2%
# 2024-10-28 :开盘未涨停：共交易【 42 】次，成功了【 27 】次，胜率【 0.6428571428571429 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 41 】次，成功了【 31 】次，胜率【 0.7560975609756098 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 44 】次，成功了【 44 】次，胜率【 1.0 %】收益率【 0 】
# 成交量增加涨停***********************
# 2024-10-28 :开盘未涨停：共交易【 41 】次，成功了【 27 】次，胜率【 0.6585365853658537 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 30 】次，成功了【 24 】次，胜率【 0.8 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 41 】次，成功了【 41 】次，胜率【 1.0 %】收益率【 0 】
# 成交额增加
# 2024-10-28 :开盘未涨停：共交易【 46 】次，成功了【 29 】次，胜率【 0.6304347826086957 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 32 】次，成功了【 25 】次，胜率【 0.78125 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 42 】次，成功了【 42 】次，胜率【 1.0 %】收益率【 0 】
# 放开所有的涨停：*****************
# 2024-10-28 :开盘未涨停：共交易【 41 】次，成功了【 27 】次，胜率【 0.6585365853658537 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 72 】次，成功了【 51 】次，胜率【 0.7083333333333334 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 101 】次，成功了【 101 】次，胜率【 1.0 %】收益率【 0 】
#  放开所有涨停，成交额 0.8 倍
# 2024-10-28 :开盘未涨停：共交易【 42 】次，成功了【 27 】次，胜率【 0.6428571428571429 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 94 】次，成功了【 66 】次，胜率【 0.7021276595744681 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 122 】次，成功了【 122 】次，胜率【 1.0 %】收益率【 0 】
#  放开所有涨停，成交额 0.9 倍  *****************
# 2024-10-28 :开盘未涨停：共交易【 42 】次，成功了【 27 】次，胜率【 0.6428571428571429 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 81 】次，成功了【 58 】次，胜率【 0.7160493827160493 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 111 】次，成功了【 111 】次，胜率【 1.0 %】收益率【 0 】
#  放开所有涨停，成交额 0.9 倍，涨幅可以低开
# 2024-10-28 :开盘未涨停：共交易【 44 】次，成功了【 27 】次，胜率【 0.6136363636363636 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 81 】次，成功了【 58 】次，胜率【 0.7160493827160493 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 111 】次，成功了【 111 】次，胜率【 1.0 %】收益率【 0 】
#  放开所有涨停，成交额 1 倍
# 2024-10-28 :开盘未涨停：共交易【 41 】次，成功了【 27 】次，胜率【 0.6585365853658537 %】收益率【 0 】
# 2024-10-28 :开盘涨停：共交易【 72 】次，成功了【 51 】次，胜率【 0.7083333333333334 %】收益率【 0 】
# 2024-10-28 :一字板：共交易【 101 】次，成功了【 101 】次，胜率【 1.0 %】收益率【 0 】
# # 放开所有的涨停200 天：
# 2024-03-13 :开盘未涨停：共交易【 150 】次，成功了【 83 】次，胜率【 0.5533333333333333 %】收益率【 0 】
# 2024-03-13 :开盘涨停：共交易【 153 】次，成功了【 102 】次，胜率【 0.6666666666666666 %】收益率【 0 】
# 2024-03-13 :一字板：共交易【 212 】次，成功了【 208 】次，胜率【 0.9811320754716981 %】收益率【 0 】

#  严格收益模式 *****************
# 最终:开盘未涨停：共交易【 41 】次，成功了【 26 】次，胜率【 0.6341463414634146 %】收益率【 1.7891974214633684 】
# 最终:开盘涨停：共交易【 75 】次，成功了【 54 】次，胜率【 0.72 %】收益率【 1.8114753467840263 】
# 最终:一字板：共交易【 102 】次，成功了【 102 】次，胜率【 1.0 %】收益率【 9.614848636253122 】
# 格局一天*****************
# 最终:开盘未涨停：共交易【 40 】次，成功了【 27 】次，胜率【 0.675 %】收益率【 1.9017615621188009 】
# 最终:开盘涨停：共交易【 75 】次，成功了【 54 】次，胜率【 0.72 %】收益率【 1.8114753467840263 】
# 最终:一字板：共交易【 99 】次，成功了【 99 】次，胜率【 1.0 %】收益率【 9.309599932020214 】
duration = 50
# 获取当前日期和时间
end_date = datetime.date.today()
tdays_list = get_trade_days(end_date=end_date.strftime('%Y-%m-%d'),count=duration)
# 假设要打印过去30天，你可以根据实际需求修改这个天数


buy_1 = 0
sell_1= 0
buy_2 = 0
sell_2= 0
buy_3 = 0
sell_3= 0
rate_1 = 0
rate_2 = 0
rate_3 = 0
for i in range(len(tdays_list)-1, 1, -1):
    current_date = tdays_list[i].strftime('%Y-%m-%d')
    
    end_date = tdays_list[len(tdays_list)-1].strftime('%Y-%m-%d')
    next_date = None
    next_next_date = None
    next_next_next_date = None
    before_before_date = None
    before_date = None
    
    
    print('**'*10,'第',i ,'轮循环','**'*10)
    print(' 当前交易日是:',current_date)
    if i-2 >= 0 and i-2 < len(tdays_list):
        before_before_date = tdays_list[i-2].strftime('%Y-%m-%d')
        print(' 前2个交易日是:',before_before_date)
        
    if i-1 >= 0 and i-1 < len(tdays_list):
        before_date = tdays_list[i-1].strftime('%Y-%m-%d')
        print(' 前1个交易日是:',before_date)
        
    if i+1 < len(tdays_list):
        next_date = tdays_list[i+1].strftime('%Y-%m-%d')
        print(' 下1个交易日是:',next_date)
    
    
    if i+2 < len(tdays_list):
        next_next_date = tdays_list[i+2].strftime('%Y-%m-%d')
        print(' 下2个交易日是:',next_next_date)
        
    if i+3 < len(tdays_list):
        next_next_next_date = tdays_list[i+3].strftime('%Y-%m-%d')
        print(' 下3个交易日是:',next_next_next_date)
    new_query = ''
    
    if before_date and current_date:
        new_query = query_1_part1 + \
                " " + format_date_to_chinese(before_date) + query_1_part2 + \
                " " + format_date_to_chinese(before_date) + query_1_part22 + \
                " " + format_date_to_chinese(current_date) + query_1_part3 + \
                " " + format_date_to_chinese(before_date) + query_1_part41+ \
                " " + format_date_to_chinese(before_date) + query_1_part42 + \
                " " + format_date_to_chinese(current_date) + query_1_part5+ \
                " " + format_date_to_chinese(current_date) + query_1_part6+ \
                " " + format_date_to_chinese(current_date) + query_1_part7
    
    print(new_query)
    count = 0
    while count<1:
        print('开始循环:',count)
        time.sleep(1)
        count+=1
        stock_cnt_1 = 0
        stock_cnt_2 = 0
        stock_cnt_3 = 0
        result = query_iwencai(new_query,domain='股票',timeout=10)
        if len(result)>0:
            result = result.sort_values(by='情绪温度环比增长', ascending=False)
            log.info(result)
            stock_code_list = result['股票代码'].tolist()
            for code in stock_code_list:
                print('-------------------当前处理股票-------------------:',code)
#                 JJ(code,current_date)
                value = get_price(securities=[code], end_date= end_date, fre_step = '1d', fields=['open', 'close', 'high', 'low'], skip_paused=True, fq='pre',  bar_count=duration+100,  is_panel=0)
            
                if before_date and before_date in  value[code].index and current_date and current_date in value[code].index and next_date and next_date in value[code].index  and next_next_date and next_next_date in value[code].index :
#                   
                    buy_flag_1 = value[code].loc[current_date]['open'] < round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_2 = value[code].loc[current_date]['open'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_3 = value[code].loc[current_date]['low'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    sell_flag_1 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['high']
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['open']*1
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['open']
                    sell_flag_1_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['high']
                    if buy_flag_1 and stock_cnt_1==0:
                        print(' 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】大于等于【',before_date,'】 的收盘价:【',value[code].loc[before_date]['close'],'】')
                        buy_1+=1
                        stock_cnt_1+=1
                        if sell_flag_2 or sell_flag_1:
                            print('开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_date]['high']/value[code].loc[current_date]['open']-1
                        elif sell_flag_2_1 or sell_flag_1_1:
                            print(' 格局一天，开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_next_date,'】 的 最高价:【',value[code].loc[next_next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_next_date]['high']/value[code].loc[current_date]['open']-1
                        else:
                            rate_1 += value[code].loc[next_next_date]['close']/value[code].loc[current_date]['open']-1
                        
#                     开盘涨停-非一字板
                    if buy_flag_2 and not buy_flag_3:
                        print('开盘涨停非一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_2+=1
                        stock_cnt_2+=1
                        if sell_flag_1:
                            print('开盘涨停非一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_2+=1
                            rate_2 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_2 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1
                        
                            
#                      开盘涨停一字板
                    if buy_flag_3 :
                        print('一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_3+=1
                        stock_cnt_3+=1
                        if sell_flag_1:
                            print('一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_3+=1
                            rate_3 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_3 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1          
            
            break
    if buy_1>0:
        print(current_date,':开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
    if buy_2>0:
        print(current_date,':开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
    if buy_3>0:
        print(current_date,':一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')
    print('------------------循环结束----------------------')



if buy_1>0:
    print('最终:',current_date,'-开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
if buy_2>0:
    print('最终:',current_date,'-开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
if buy_3>0:
    print('最终:',current_date,'-一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')


******************** 第 49 轮循环 ********************
 当前交易日是: 2025-01-09
 前2个交易日是: 2025-01-07
 前1个交易日是: 2025-01-08
沪深主板排除ST，股票市值20-500亿之间； 2025年01月08日成交额大于前一日成交额的0.9倍； 2025年01月08日涨停； 2025年01月09日竞价强度排名； 2025年01月08日情绪温度相比前一日提升排名； 2025年01月08日情绪温度； 2025年01月09日竞价金额与昨日最大一分钟成交额占比小于150%； 2025年01月09日竞价评级是看多； 2025年01月09日开盘价大于前一日收盘价的1.02倍；
开始循环: 0
------------------循环结束----------------------
******************** 第 48 轮循环 ********************
 当前交易日是: 2025-01-08
 前2个交易日是: 2025-01-06
 前1个交易日是: 2025-01-07
 下1个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月07日成交额大于前一日成交额的0.9倍； 2025年01月07日涨停； 2025年01月08日竞价强度排名； 2025年01月07日情绪温度相比前一日提升排名； 2025年01月07日情绪温度； 2025年01月08日竞价金额与昨日最大一分钟成交额占比小于150%； 2025年01月08日竞价评级是看多； 2025年01月08日开盘价大于前一日收盘价的1.02倍；
开始循环: 0
-------------------当前处理股票-------------------: 002418.SZ
-------------------当前处理股票-------------------: 002366.SZ
-------------------当前处理股票-------------------: 600105.SH
-------------------当前处理股票-------------------: 000573.SZ


2025-01-09 01:58:22.592343 - INFO -    {(}开盘价:不复权{/}收盘价:不复权{)}  {(}成交额{/}成交额{)} 开盘价:不复权           总市值     情绪温度  \
1                 1.101307         2.293489    3.37  3.829668e+09  86.9983   
3                 1.072961         1.281994       5  9.884454e+09  32.7080   
2                 1.032454         2.109069    5.09  7.309974e+09  24.0473   
0                 1.101205        10.319094    4.57  2.916942e+09  55.9466   

   情绪温度环比增长 情绪温度环比增长排名  情绪温度环比增长排名名次  情绪温度环比增长排名基数           成交额  收盘价:不复权  涨停  \
1   32.6412    50/5395            50          5395  1.75784116E8     3.06  涨停   
3   28.5532    67/5395            67          5395  1.47995964E8     4.66  涨停   
2   21.9833    94/5395            94          5395  1.50224415E8     4.93  涨停   
0    9.2720   208/5395           208          5395  3.42762142E7     4.15  涨停   

        竞价强度   竞价强度排名  竞价强度排名名次  竞价强度排名基数  竞价金额与昨日最大一分钟成交额占比       股票代码  \
1  10.317576   4/5395         4      5395          39.453261  002418.SZ   
3   9.705640   7/

------------------循环结束----------------------
******************** 第 47 轮循环 ********************
 当前交易日是: 2025-01-07
 前2个交易日是: 2025-01-03
 前1个交易日是: 2025-01-06
 下1个交易日是: 2025-01-08
 下2个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月06日成交额大于前一日成交额的0.9倍； 2025年01月06日涨停； 2025年01月07日竞价强度排名； 2025年01月06日情绪温度相比前一日提升排名； 2025年01月06日情绪温度； 2025年01月07日竞价金额与昨日最大一分钟成交额占比小于150%； 2025年01月07日竞价评级是看多； 2025年01月07日开盘价大于前一日收盘价的1.02倍；
开始循环: 0


2025-01-09 01:58:24.128801 - INFO -    {(}开盘价:不复权{/}收盘价:不复权{)}  {(}成交额{/}成交额{)} 开盘价:不复权           总市值     情绪温度  \
1                 1.100086         0.918727   12.75  3.094134e+09  77.0340   
0                 1.024390         2.214058    5.88  3.781159e+09  39.7248   

   情绪温度环比增长 情绪温度环比增长排名  情绪温度环比增长排名名次  情绪温度环比增长排名基数           成交额  收盘价:不复权  涨停  \
1   64.6965     9/5395             9          5395  1.47633709E8    11.59  涨停   
0   34.3792    46/5395            46          5395  5.42719006E8     5.74  涨停   

       竞价强度   竞价强度排名  竞价强度排名名次  竞价强度排名基数  竞价金额与昨日最大一分钟成交额占比       股票代码 股票市场类型  \
1  9.436002  12/5395        12      5395         116.119261  603269.SH   沪深主板   
0  8.743792  35/5395        35      5395          40.604315  600405.SH   沪深主板   

   股票简称 集合竞价评级  
1  海鸥股份     看多  
0   动力源     看多  


-------------------当前处理股票-------------------: 603269.SH
-------------------当前处理股票-------------------: 600405.SH
------------------循环结束----------------------
******************** 第 46 轮循环 ********************
 当前交易日是: 2025-01-06
 前2个交易日是: 2025-01-02
 前1个交易日是: 2025-01-03
 下1个交易日是: 2025-01-07
 下2个交易日是: 2025-01-08
 下3个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月03日成交额大于前一日成交额的0.9倍； 2025年01月03日涨停； 2025年01月06日竞价强度排名； 2025年01月03日情绪温度相比前一日提升排名； 2025年01月03日情绪温度； 2025年01月06日竞价金额与昨日最大一分钟成交额占比小于150%； 2025年01月06日竞价评级是看多； 2025年01月06日开盘价大于前一日收盘价的1.02倍；
开始循环: 0


...

开盘涨停非一字板：买入买入买入!!!!  该股票在【 2024-11-06 】 的 开盘价:【 5.93 】等于【 2024-11-05 】 的收盘价的1.1倍:【 5.39 】
开盘涨停非一字板：卖出卖出卖出!!!! 该股票在【 2024-11-07 】 的 最高价:【 6.52 】大于等于【 2024-11-06 】 的最高价:【 5.93 】
-------------------当前处理股票-------------------: 001319.SZ
-------------------当前处理股票-------------------: 000955.SZ
2024-11-06 :开盘未涨停：共交易【 37 】次，成功了【 28 】次，胜率【 0.7567567567567568 %】收益率【 1.880954872429016 】
2024-11-06 :开盘涨停：共交易【 57 】次，成功了【 38 】次，胜率【 0.6666666666666666 %】收益率【 0.9832746714685524 】
2024-11-06 :一字板：共交易【 87 】次，成功了【 87 】次，胜率【 1.0 %】收益率【 8.112088367371669 】
------------------循环结束----------------------
******************** 第 3 轮循环 ********************
 当前交易日是: 2024-11-05
 前2个交易日是: 2024-11-01
 前1个交易日是: 2024-11-04
 下1个交易日是: 2024-11-06
 下2个交易日是: 2024-11-07
 下3个交易日是: 2024-11-08
沪深主板排除ST，股票市值20-500亿之间； 2024年11月04日成交额大于前一日成交额的0.9倍； 2024年11月04日涨停； 2024年11月05日竞价强度排名； 2024年11月04日情绪温度相比前一日提升排名； 2024年11月04日情绪温度； 2024年11月05日竞价金额与昨日最大一分钟成交额占比小于150%； 2024年11月05日竞价评级是看多； 2024年11月05日开盘价大于前一日收盘价的1.02倍；
开始循环: 0
---------------

2025-01-09 01:59:53.102770 - INFO -    {(}开盘价:不复权{/}收盘价:不复权{)}  {(}成交额{/}成交额{)} 开盘价:不复权           总市值     情绪温度  \
2                 1.042735         1.361670    3.66  3.767204e+09  47.5096   
4                 1.022143         0.979285   14.31  5.141207e+09  90.7787   
3                 1.071111         1.290938   36.15  3.721280e+09  34.3931   
0                 1.080074         1.973487     5.8  5.538148e+09  43.0832   
5                 1.035782         0.936563    16.5  2.040781e+09  19.6863   
1                 1.031887         1.757750    23.3  2.561484e+09  15.8034   

   情绪温度环比增长 情绪温度环比增长排名  情绪温度环比增长排名名次  情绪温度环比增长排名基数           成交额  收盘价:不复权  涨停  \
2   37.0943    57/5395            57          5395   2.2543878E8     3.51  涨停   
4   35.6137    62/5395            62          5395  2.56585926E9    14.00  涨停   
3   30.7313    84/5395            84          5395  1.03369297E8    33.75  涨停   
0   24.5725   114/5395           114          5395   3.5822766E8     5.37  涨停   
5   17.0852 

2024-11-05 :开盘未涨停：共交易【 38 】次，成功了【 29 】次，胜率【 0.7631578947368421 %】收益率【 2.0421570582213655 】
2024-11-05 :开盘涨停：共交易【 57 】次，成功了【 38 】次，胜率【 0.6666666666666666 %】收益率【 0.9832746714685524 】
2024-11-05 :一字板：共交易【 87 】次，成功了【 87 】次，胜率【 1.0 %】收益率【 8.112088367371669 】
------------------循环结束----------------------
******************** 第 2 轮循环 ********************
 当前交易日是: 2024-11-04
 前2个交易日是: 2024-10-31
 前1个交易日是: 2024-11-01
 下1个交易日是: 2024-11-05
 下2个交易日是: 2024-11-06
 下3个交易日是: 2024-11-07
沪深主板排除ST，股票市值20-500亿之间； 2024年11月01日成交额大于前一日成交额的0.9倍； 2024年11月01日涨停； 2024年11月04日竞价强度排名； 2024年11月01日情绪温度相比前一日提升排名； 2024年11月01日情绪温度； 2024年11月04日竞价金额与昨日最大一分钟成交额占比小于150%； 2024年11月04日竞价评级是看多； 2024年11月04日开盘价大于前一日收盘价的1.02倍；
开始循环: 0
-------------------当前处理股票-------------------: 600366.SH
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2024-11-04 】 的 开盘价:【 8.3 】大于等于【 2024-11-01 】 的收盘价:【 7.68 】
开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【 2024-11-05 】 的 最高价:【 8.8 】大于等于【 2024-11-04 】 的开盘价:【 8.3 】
-------------------当前处理股票-------------------: 605266.SH
---------------

2025-01-09 01:59:55.083148 - INFO -    {(}开盘价:不复权{/}收盘价:不复权{)}  {(}成交额{/}成交额{)} 开盘价:不复权           总市值      情绪温度  \
2                 1.080729         2.147410     8.3  7.880124e+09   55.2063   
1                 1.085371         2.837021    25.3  3.329042e+09   42.6885   
5                 1.070495         1.379174   18.83  4.821561e+09   44.3605   
8                 1.100047         1.064829   23.42  1.182147e+10   52.3801   
7                 1.101449         1.081836     3.8  4.129705e+09   23.7351   
3                 1.088977         1.782678    24.6  4.805041e+09   36.4900   
4                 1.101093         1.716331    4.03  2.146029e+10  100.0000   
0                 1.099602         3.547700    2.76  9.471203e+09   92.9752   
6                 1.100124         1.262921   17.69  1.702773e+10  100.0000   

   情绪温度环比增长 情绪温度环比增长排名  情绪温度环比增长排名名次  情绪温度环比增长排名基数           成交额  收盘价:不复权  涨停  \
2   50.4618    23/5395            23          5395  4.46682838E8     7.68  涨停   
1   37.9899

 603666.SH
-------------------当前处理股票-------------------: 000415.SZ
一字板：买入买入买入!!!!  该股票在【 2024-11-04 】 的 开盘价:【 4.03 】等于【 2024-11-01 】 的收盘价的1.1倍:【 3.66 】
一字板：卖出卖出卖出!!!! 该股票在【 2024-11-05 】 的 最高价:【 4.43 】大于等于【 2024-11-04 】 的最高价:【 4.03 】
-------------------当前处理股票-------------------: 600340.SH
一字板：买入买入买入!!!!  该股票在【 2024-11-04 】 的 开盘价:【 2.76 】等于【 2024-11-01 】 的收盘价的1.1倍:【 2.51 】
一字板：卖出卖出卖出!!!! 该股票在【 2024-11-05 】 的 最高价:【 3.04 】大于等于【 2024-11-04 】 的最高价:【 2.76 】
-------------------当前处理股票-------------------: 002416.SZ
一字板：买入买入买入!!!!  该股票在【 2024-11-04 】 的 开盘价:【 17.69 】等于【 2024-11-01 】 的收盘价的1.1倍:【 16.08 】
一字板：卖出卖出卖出!!!! 该股票在【 2024-11-05 】 的 最高价:【 19.46 】大于等于【 2024-11-04 】 的最高价:【 17.69 】
2024-11-04 :开盘未涨停：共交易【 39 】次，成功了【 30 】次，胜率【 0.7692307692307693 %】收益率【 2.1023980220767875 】
2024-11-04 :开盘涨停：共交易【 59 】次，成功了【 40 】次，胜率【 0.6779661016949152 %】收益率【 1.0831892743720535 】
2024-11-04 :一字板：共交易【 90 】次，成功了【 90 】次，胜率【 1.0 %】收益率【 8.412849754973031 】
------------------循环结束----------------------
最终: 2024-11-04 -开盘未涨

##  炸板弱转强

In [60]:
import datetime
import pandas as pd
import time
import numpy as np
from tabulate import tabulate
from mindgo_api import *

def format_date_to_chinese(date_str):
    """
    将格式为%Y-%m-%d的日期字符串转换为中文日期格式（%Y年%m月%d日）
    """
    date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    return date_obj.strftime('%Y年%m月%d日')


pd.options.display.max_columns = None  # 显示所有列
pd.options.display.max_rows = None  # 显示所有行
  
part1 = "沪深主板排除ST，股票市值20-500亿之间；"
part2 = "没有涨停；"
part3 = "收盘价大于五日均线；"
part4 = "量能大于前一日；"
part5 = "涨幅大于3%小于8%；"
part6 = "涨停炸板；"
part7 = "情绪温度排名大于等于前一日；"
part8 = "情绪温度大于10分；"
part9 = "主力资金大于0；"
part10 = "主力进攻；"

# 回测 50 天
# 最终: 2024-11-04 -开盘未涨停：共交易【 26 】次，成功了【 25 】次，胜率【 0.9615384615384616 %】收益率【 1.136712294800141 】
# 最终: 2024-11-04 -开盘涨停：共交易【 1 】次，成功了【 1 】次，胜率【 1.0 %】收益率【 0.0 】
#  回测 200 天
# ------------------循环结束----------------------
# 最终: 2024-03-20 -开盘未涨停：共交易【 85 】次，成功了【 64 】次，胜率【 0.7529411764705882 %】收益率【 1.552363837000062 】
# 最终: 2024-03-20 -开盘涨停：共交易【 2 】次，成功了【 2 】次，胜率【 1.0 %】收益率【 0.06083650190114076 】
duration = 200
# 获取当前日期和时间
end_date = datetime.date.today()
tdays_list = get_trade_days(end_date=end_date.strftime('%Y-%m-%d'),count=duration)
# 假设要打印过去30天，你可以根据实际需求修改这个天数


buy_1 = 0
sell_1= 0
buy_2 = 0
sell_2= 0
buy_3 = 0
sell_3= 0
rate_1 = 0
rate_2 = 0
rate_3 = 0
for i in range(len(tdays_list)-1, 1, -1):
    current_date = tdays_list[i].strftime('%Y-%m-%d')
    
    end_date = tdays_list[len(tdays_list)-1].strftime('%Y-%m-%d')
    next_date = None
    next_next_date = None
    next_next_next_date = None
    before_before_date = None
    before_date = None
    
    
    print('**'*10,'第',i ,'轮循环','**'*10)
    print(' 当前交易日是:',current_date)
    if i-2 >= 0 and i-2 < len(tdays_list):
        before_before_date = tdays_list[i-2].strftime('%Y-%m-%d')
        print(' 前2个交易日是:',before_before_date)
        
    if i-1 >= 0 and i-1 < len(tdays_list):
        before_date = tdays_list[i-1].strftime('%Y-%m-%d')
        print(' 前1个交易日是:',before_date)
        
    if i+1 < len(tdays_list):
        next_date = tdays_list[i+1].strftime('%Y-%m-%d')
        print(' 下1个交易日是:',next_date)
    
    
    if i+2 < len(tdays_list):
        next_next_date = tdays_list[i+2].strftime('%Y-%m-%d')
        print(' 下2个交易日是:',next_next_date)
        
    if i+3 < len(tdays_list):
        next_next_next_date = tdays_list[i+3].strftime('%Y-%m-%d')
        print(' 下3个交易日是:',next_next_next_date)
    new_query = ''
    
    if before_date and current_date:
        new_query = part1 +  \
        " " + format_date_to_chinese(before_before_date) + part2 +  \
        " " + format_date_to_chinese(before_date) + part3 +  \
        " " + format_date_to_chinese(before_date) + part4+  \
        " " + format_date_to_chinese(before_date) + part5 +  \
        " " + format_date_to_chinese(before_date) + part6+  \
        " " + format_date_to_chinese(before_date) + part7+  \
        " " + format_date_to_chinese(before_date) + part8 +  \
        " " + format_date_to_chinese(before_date) + part9 +  \
        " " + format_date_to_chinese(before_date) + part10
    
    print(new_query)
    count = 0
    while count<1:
        print('开始循环:',count)
        time.sleep(1)
        count+=1
        stock_cnt_1 = 0
        stock_cnt_2 = 0
        stock_cnt_3 = 0
        result = query_iwencai(new_query,domain='股票',timeout=10)
        if len(result)>0:
            result = result.sort_values(by='主力资金流向', ascending=False)
            log.info(result)
            stock_code_list = result['股票代码'].tolist()
            for code in stock_code_list:
                print('-------------------当前处理股票-------------------:',code)
#                 JJ(code,current_date)
                value = get_price(securities=[code], end_date= end_date, fre_step = '1d', fields=['open', 'close', 'high', 'low'], skip_paused=True, fq='pre',  bar_count=duration+100,  is_panel=0)
            
                if before_date and before_date in  value[code].index and current_date and current_date in value[code].index and next_date and next_date in value[code].index  and next_next_date and next_next_date in value[code].index :
#                   
                    buy_flag_1 = value[code].loc[current_date]['open'] < round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_2 = value[code].loc[current_date]['open'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_3 = value[code].loc[current_date]['low'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    sell_flag_1 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['high']
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['open']*1
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['open']
                    sell_flag_1_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['high']
                    if buy_flag_1 and stock_cnt_1==0:
                        print(' 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】大于等于【',before_date,'】 的收盘价:【',value[code].loc[before_date]['close'],'】')
                        buy_1+=1
                        stock_cnt_1+=1
                        if sell_flag_2 or sell_flag_1:
                            print('开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_date]['high']/value[code].loc[current_date]['open']-1
                        elif sell_flag_2_1 or sell_flag_1_1:
                            print(' 格局一天，开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_next_date,'】 的 最高价:【',value[code].loc[next_next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_next_date]['high']/value[code].loc[current_date]['open']-1
                        else:
                            rate_1 += value[code].loc[next_next_date]['close']/value[code].loc[current_date]['open']-1
                        
#                     开盘涨停-非一字板
                    if buy_flag_2 and not buy_flag_3:
                        print('开盘涨停非一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_2+=1
                        stock_cnt_2+=1
                        if sell_flag_1:
                            print('开盘涨停非一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_2+=1
                            rate_2 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_2 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1
                        
                            
#                      开盘涨停一字板
                    if buy_flag_3 :
                        print('一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_3+=1
                        stock_cnt_3+=1
                        if sell_flag_1:
                            print('一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_3+=1
                            rate_3 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_3 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1          
            
            break
    if buy_1>0:
        print(current_date,':开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
    if buy_2>0:
        print(current_date,':开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
    if buy_3>0:
        print(current_date,':一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')
    print('------------------循环结束----------------------')



if buy_1>0:
    print('最终:',current_date,'-开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
if buy_2>0:
    print('最终:',current_date,'-开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
if buy_3>0:
    print('最终:',current_date,'-一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')


******************** 第 199 轮循环 ********************
 当前交易日是: 2025-01-09
 前2个交易日是: 2025-01-07
 前1个交易日是: 2025-01-08
沪深主板排除ST，股票市值20-500亿之间； 2025年01月07日没有涨停； 2025年01月08日收盘价大于五日均线； 2025年01月08日量能大于前一日； 2025年01月08日涨幅大于3%小于8%； 2025年01月08日涨停炸板； 2025年01月08日情绪温度排名大于等于前一日； 2025年01月08日情绪温度大于10分； 2025年01月08日主力资金大于0； 2025年01月08日主力进攻；
开始循环: 0
------------------循环结束----------------------
******************** 第 198 轮循环 ********************
 当前交易日是: 2025-01-08
 前2个交易日是: 2025-01-06
 前1个交易日是: 2025-01-07
 下1个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月06日没有涨停； 2025年01月07日收盘价大于五日均线； 2025年01月07日量能大于前一日； 2025年01月07日涨幅大于3%小于8%； 2025年01月07日涨停炸板； 2025年01月07日情绪温度排名大于等于前一日； 2025年01月07日情绪温度大于10分； 2025年01月07日主力资金大于0； 2025年01月07日主力进攻；
开始循环: 0
------------------循环结束----------------------
******************** 第 197 轮循环 ********************
 当前交易日是: 2025-01-07
 前2个交易日是: 2025-01-03
 前1个交易日是: 2025-01-06
 下1个交易日是: 2025-01-08
 下2个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月03日没有涨停； 2025年01月06日收盘价大于五日均线； 2025年01月06

2025-01-09 03:05:50.711264 - INFO -             vol  {(}1日vol{-}1日vol{)}  {(}情绪温度{-}情绪温度{)}  {(}行情收盘价{-}5日均线{)}  \
1  11332700.000           27597069.0            30.5743               0.734   
0  15275155.000           21404684.0            14.0702               0.274   

   主力动向      主力资金流向      均线           总市值     情绪温度  曾涨停     涨跌幅:前复权  \
1     1  32150413.0  19.896  9.349619e+09  35.6069  曾涨停  5.63236047   
0     1   4952666.0  13.546  6.013213e+09  18.2115  曾涨停  4.85584219   

        股票代码 股票市场类型  股票简称   行情收盘价  
1  600645.SH   沪深主板  中源协和  20.630  
0  003040.SZ   沪深主板   楚天龙  13.820  


-------------------当前处理股票-------------------: 600645.SH
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2025-01-06 】 的 开盘价:【 20.2 】大于等于【 2025-01-03 】 的收盘价:【 20.63 】
开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【 2025-01-07 】 的 最高价:【 20.92 】大于等于【 2025-01-06 】 的开盘价:【 20.2 】
-------------------当前处理股票-------------------: 003040.SZ
2025-01-06 :开盘未涨停：共交易【 1 】次，成功了【 1 】次，胜率【 1.0 %】收益率【 0.035643564356435675 】
------------------循环结束----------------------
******************** 第 195 轮循环 ********************
 当前交易日是: 2025-01-03
 前2个交易日是: 2024-12-31
 前1个交易日是: 2025-01-02
 下1个交易日是: 2025-01-06
 下2个交易日是: 2025-01-07
 下3个交易日是: 2025-01-08
沪深主板排除ST，股票市值20-500亿之间； 2024年12月31日没有涨停； 2025年01月02日收盘价大于五日均线； 2025年01月02日量能大于前一日； 2025年01月02日涨幅大于3%小于8%； 2025年01月02日涨停炸板； 2025年01月02日情绪温度排名大于等于前一日； 2025年01月02日情绪温度大于10分； 2025年01月02日主力资金大于0； 2025年01月02日主力进攻；
开始循环: 0


2025-01-09 03:05:52.131526 - INFO -             vol  {(}1日vol{-}1日vol{)}  {(}情绪温度{-}情绪温度{)}  {(}行情收盘价{-}5日均线{)}  \
2  48398208.000           27236530.0            31.5209               2.242   
0  47881220.000           37288250.0             9.0957               0.168   
1  11986628.000            6662140.0            15.0847               1.352   

   主力动向      主力资金流向      均线           总市值     情绪温度  曾涨停     涨跌幅:前复权  \
2     1  79368320.0  67.438  2.182861e+10  42.1604  曾涨停  7.86377709   
0     1  11615376.0   4.682  4.583019e+09  10.0808  曾涨停  7.30088496   
1     1   1876007.0  25.028  2.896147e+09  19.9451  曾涨停  5.26735834   

        股票代码 股票市场类型  股票简称   行情收盘价  
2  603662.SH   沪深主板  柯力传感  69.680  
0  600698.SH   沪深主板  湖南天雁   4.850  
1  605289.SH   沪深主板  罗曼股份  26.380  


-------------------当前处理股票-------------------: 603662.SH
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2025-01-03 】 的 开盘价:【 69.59 】大于等于【 2025-01-02 】 的收盘价:【 69.68 】
开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【 2025-01-06 】 的 最高价:【 70.88 】大于等于【 2025-01-03 】 的开盘价:【 69.59 】
-------------------当前处理股票-------------------: 600698.SH
-------------------当前处理股票-------------------: 605289.SH
2025-01-03 :开盘未涨停：共交易【 2 】次，成功了【 2 】次，胜率【 1.0 %】收益率【 0.054180710498122675 】
------------------循环结束----------------------
******************** 第 194 轮循环 ********************
 当前交易日是: 2025-01-02
 前2个交易日是: 2024-12-30
 前1个交易日是: 2024-12-31
 下1个交易日是: 2025-01-03
 下2个交易日是: 2025-01-06
 下3个交易日是: 2025-01-07
沪深主板排除ST，股票市值20-500亿之间； 2024年12月30日没有涨停； 2024年12月31日收盘价大于五日均线； 2024年12月31日量能大于前一日； 2024年12月31日涨幅大于3%小于8%； 2024年12月31日涨停炸板； 2024年12月31日情绪温度排名大于等于前一日； 2024年12月31日情绪温度大于10分； 2024年12月31日主力资金大于0； 2024年12月31日主力进攻；
开始循环: 0


...

-------------------当前处理股票-------------------: 603660.SH
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2024-03-26 】 的 开盘价:【 8.34 】大于等于【 2024-03-25 】 的收盘价:【 8.66 】
2024-03-26 :开盘未涨停：共交易【 83 】次，成功了【 62 】次，胜率【 0.7469879518072289 %】收益率【 1.4748206407393614 】
2024-03-26 :开盘涨停：共交易【 2 】次，成功了【 2 】次，胜率【 1.0 %】收益率【 0.06083650190114076 】
------------------循环结束----------------------
******************** 第 5 轮循环 ********************
 当前交易日是: 2024-03-25
 前2个交易日是: 2024-03-21
 前1个交易日是: 2024-03-22
 下1个交易日是: 2024-03-26
 下2个交易日是: 2024-03-27
 下3个交易日是: 2024-03-28
沪深主板排除ST，股票市值20-500亿之间； 2024年03月21日没有涨停； 2024年03月22日收盘价大于五日均线； 2024年03月22日量能大于前一日； 2024年03月22日涨幅大于3%小于8%； 2024年03月22日涨停炸板； 2024年03月22日情绪温度排名大于等于前一日； 2024年03月22日情绪温度大于10分； 2024年03月22日主力资金大于0； 2024年03月22日主力进攻；
开始循环: 0
2024-03-25 :开盘未涨停：共交易【 83 】次，成功了【 62 】次，胜率【 0.7469879518072289 %】收益率【 1.4748206407393614 】
2024-03-25 :开盘涨停：共交易【 2 】次，成功了【 2 】次，胜率【 1.0 %】收益率【 0.06083650190114076 】
------------------循环结束----------------------
******************** 第 4 轮循环 **************

2025-01-09 03:10:47.592380 - INFO -              vol  {(}1日vol{-}1日vol{)}  {(}情绪温度{-}情绪温度{)}  {(}行情收盘价{-}5日均线{)}  \
0  127678696.000           94712876.0            14.2165               0.138   

   主力动向     主力资金流向     均线           总市值    情绪温度  曾涨停     涨跌幅:前复权       股票代码  \
0     1  8833516.0  2.492  5.834888e+09  2.6161  曾涨停  6.47773279  002175.SZ   

  股票市场类型  股票简称  行情收盘价  
0   沪深主板  东方智造  2.630  


-------------------当前处理股票-------------------: 002175.SZ
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2024-03-21 】 的 开盘价:【 2.64 】大于等于【 2024-03-20 】 的收盘价:【 2.63 】
开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【 2024-03-22 】 的 最高价:【 2.68 】大于等于【 2024-03-21 】 的开盘价:【 2.64 】
2024-03-21 :开盘未涨停：共交易【 84 】次，成功了【 63 】次，胜率【 0.75 %】收益率【 1.4899721558908765 】
2024-03-21 :开盘涨停：共交易【 2 】次，成功了【 2 】次，胜率【 1.0 %】收益率【 0.06083650190114076 】
------------------循环结束----------------------
******************** 第 2 轮循环 ********************
 当前交易日是: 2024-03-20
 前2个交易日是: 2024-03-18
 前1个交易日是: 2024-03-19
 下1个交易日是: 2024-03-21
 下2个交易日是: 2024-03-22
 下3个交易日是: 2024-03-25
沪深主板排除ST，股票市值20-500亿之间； 2024年03月18日没有涨停； 2024年03月19日收盘价大于五日均线； 2024年03月19日量能大于前一日； 2024年03月19日涨幅大于3%小于8%； 2024年03月19日涨停炸板； 2024年03月19日情绪温度排名大于等于前一日； 2024年03月19日情绪温度大于10分； 2024年03月19日主力资金大于0； 2024年03月19日主力进攻；
开始循环: 0


2025-01-09 03:10:49.056966 - INFO -              vol  {(}1日vol{-}1日vol{)}  {(}情绪温度{-}情绪温度{)}  {(}行情收盘价{-}5日均线{)}  \
0  194436020.000          129852515.0            71.5189               0.544   

   主力动向       主力资金流向      均线           总市值     情绪温度  曾涨停     涨跌幅:前复权  \
0     1  148762480.0  11.146  1.866250e+10  79.9958  曾涨停  6.20629371   

        股票代码 股票市场类型  股票简称   行情收盘价  
0  002624.SZ   沪深主板  完美世界  11.690  


-------------------当前处理股票-------------------: 002624.SZ
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2024-03-20 】 的 开盘价:【 11.54 】大于等于【 2024-03-19 】 的收盘价:【 11.69 】
开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【 2024-03-21 】 的 最高价:【 12.26 】大于等于【 2024-03-20 】 的开盘价:【 11.54 】
2024-03-20 :开盘未涨停：共交易【 85 】次，成功了【 64 】次，胜率【 0.7529411764705882 %】收益率【 1.552363837000062 】
2024-03-20 :开盘涨停：共交易【 2 】次，成功了【 2 】次，胜率【 1.0 %】收益率【 0.06083650190114076 】
------------------循环结束----------------------
最终: 2024-03-20 -开盘未涨停：共交易【 85 】次，成功了【 64 】次，胜率【 0.7529411764705882 %】收益率【 1.552363837000062 】
最终: 2024-03-20 -开盘涨停：共交易【 2 】次，成功了【 2 】次，胜率【 1.0 %】收益率【 0.06083650190114076 】


## 倍量过左峰

In [43]:
import datetime
import pandas as pd
import time
import numpy as np
from tabulate import tabulate
from mindgo_api import *

def format_date_to_chinese(date_str):
    """
    将格式为%Y-%m-%d的日期字符串转换为中文日期格式（%Y年%m月%d日）
    """
    date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    return date_obj.strftime('%Y年%m月%d日')


pd.options.display.max_columns = None  # 显示所有列
pd.options.display.max_rows = None  # 显示所有行
  
query_2_part1 = "沪深主板排除ST，股票市值20-500亿之间；"
query_2_part2 = "涨停；"
query_2_part3 = "情绪温度大于50分；"
query_2_part4 = "过左峰并涨停；"
query_2_part5 = "竞价强度排名；"
#  竞价就买***************
# 最终: 2024-11-04 -开盘未涨停：共交易【 30 】次，成功了【 21 】次，胜率【 0.7 %】收益率【 1.0009295413828694 】
# 最终: 2024-11-04 -开盘涨停：共交易【 6 】次，成功了【 5 】次，胜率【 0.8333333333333334 %】收益率【 0.18081866604722008 】
# 最终: 2024-11-04 -一字板：共交易【 15 】次，成功了【 15 】次，胜率【 1.0 %】收益率【 1.4567642331612192 】
# 格局一天和不格局没区别，选择不格局
# 回测 150 天
# ------------------循环结束----------------------
# 最终:开盘未涨停：共交易【 152 】次，成功了【 103 】次，胜率【 0.6776315789473685 %】收益率【 4.379856990479235 】
# 最终:开盘涨停：共交易【 31 】次，成功了【 26 】 次，胜率【 0.8387096774193549 %】收益率【 1.3862351279155263 】
# 最终:一字板：共交易【 42 】次，成功了【 42 】次，胜率【 1.0 %】收益率【 4.109534036859116 】
duration = 50
# 获取当前日期和时间
end_date = datetime.date.today()
tdays_list = get_trade_days(end_date=end_date.strftime('%Y-%m-%d'),count=duration)
# 假设要打印过去30天，你可以根据实际需求修改这个天数




buy_1 = 0
sell_1= 0
buy_2 = 0
sell_2= 0
buy_3 = 0
sell_3= 0
rate_1 = 0
rate_2 = 0
rate_3 = 0
for i in range(len(tdays_list)-1, 1, -1):
    current_date = tdays_list[i].strftime('%Y-%m-%d')
    
    end_date = tdays_list[len(tdays_list)-1].strftime('%Y-%m-%d')
    next_date = None
    next_next_date = None
    next_next_next_date = None
    before_before_date = None
    before_date = None
    
    
    print('**'*10,'第',i ,'轮循环','**'*10)
    print(' 当前交易日是:',current_date)
    if i-2 >= 0 and i-2 < len(tdays_list):
        before_before_date = tdays_list[i-2].strftime('%Y-%m-%d')
        print(' 前2个交易日是:',before_before_date)
        
    if i-1 >= 0 and i-1 < len(tdays_list):
        before_date = tdays_list[i-1].strftime('%Y-%m-%d')
        print(' 前1个交易日是:',before_date)
        
    if i+1 < len(tdays_list):
        next_date = tdays_list[i+1].strftime('%Y-%m-%d')
        print(' 下1个交易日是:',next_date)
    
    
    if i+2 < len(tdays_list):
        next_next_date = tdays_list[i+2].strftime('%Y-%m-%d')
        print(' 下2个交易日是:',next_next_date)
    
    
    if i+3 < len(tdays_list):
        next_next_next_date = tdays_list[i+3].strftime('%Y-%m-%d')
        print(' 下3个交易日是:',next_next_next_date)
    
    new_query = ''
    if before_before_date and before_date:
        new_query = query_2_part1 +  \
            " " + format_date_to_chinese(before_before_date) + query_2_part2 +  \
            " " + format_date_to_chinese(before_date) + query_2_part3 + \
            " " + format_date_to_chinese(before_date) + query_2_part4+ \
            " " + format_date_to_chinese(before_date) + query_2_part5

     
    print(new_query)
    count = 0
    while count<1:
        print('开始循环:',count)
        time.sleep(1)
        count+=1
        stock_cnt_1 = 0
        stock_cnt_2 = 0
        stock_cnt_3 = 0
        result = query_iwencai(new_query,domain='股票',timeout=10)
        if len(result)>0:
#             result = result.sort_values(by='情绪温度环比增长', ascending=False)
            log.info(result)
            stock_code_list = result['股票代码'].tolist()
            for code in stock_code_list:
                print('-------------------当前处理股票-------------------:',code)
#                 JJ(code,current_date)
                value = get_price(securities=[code], end_date= end_date, fre_step = '1d', fields=['open', 'close', 'high', 'low'], skip_paused=True, fq='pre',  bar_count=duration+100,  is_panel=0)
            
                if before_date and before_date in  value[code].index and current_date and current_date in value[code].index and next_date and next_date in value[code].index  and next_next_date and next_next_date in value[code].index :
#                   
                    buy_flag_1 = value[code].loc[current_date]['open'] < round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_2 = value[code].loc[current_date]['open'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_3 = value[code].loc[current_date]['low'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    sell_flag_1 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['high']
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['open']*1
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['open']
                    sell_flag_1_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['high']
                    if buy_flag_1 and stock_cnt_1==0:
                        print(' 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】大于等于【',before_date,'】 的收盘价:【',value[code].loc[before_date]['close'],'】')
                        buy_1+=1
                        stock_cnt_1+=1
                        if sell_flag_2 or sell_flag_1:
                            print('开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_date]['high']/value[code].loc[current_date]['open']-1
                        elif sell_flag_2_1 or sell_flag_1_1:
                            print(' 格局一天，开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_next_date,'】 的 最高价:【',value[code].loc[next_next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_next_date]['high']/value[code].loc[current_date]['open']-1
                        else:
                            rate_1 += value[code].loc[next_next_date]['close']/value[code].loc[current_date]['open']-1
                        
#                     开盘涨停-非一字板
                    if buy_flag_2 and not buy_flag_3:
                        print('开盘涨停非一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_2+=1
                        stock_cnt_2+=1
                        if sell_flag_1:
                            print('开盘涨停非一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_2+=1
                            rate_2 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_2 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1
                        
                            
#                      开盘涨停一字板
                    if buy_flag_3 :
                        print('一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_3+=1
                        stock_cnt_3+=1
                        if sell_flag_1:
                            print('一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_3+=1
                            rate_3 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_3 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1
            break
    if buy_1>0:
        print(next_date,':开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
    if buy_2>0:
        print(next_date,':开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
    if buy_3>0:
        print(next_date,':一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')
    print('------------------循环结束----------------------')



if buy_1>0:
    print('最终:',current_date,'-开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
if buy_2>0:
    print('最终:',current_date,'-开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
if buy_3>0:
    print('最终:',current_date,'-一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')


******************** 第 49 轮循环 ********************
 当前交易日是: 2025-01-09
 前2个交易日是: 2025-01-07
 前1个交易日是: 2025-01-08
沪深主板排除ST，股票市值20-500亿之间； 2025年01月07日涨停； 2025年01月08日情绪温度大于50分； 2025年01月08日过左峰并涨停； 2025年01月08日竞价强度排名；
开始循环: 0
------------------循环结束----------------------
******************** 第 48 轮循环 ********************
 当前交易日是: 2025-01-08
 前2个交易日是: 2025-01-06
 前1个交易日是: 2025-01-07
 下1个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月06日涨停； 2025年01月07日情绪温度大于50分； 2025年01月07日过左峰并涨停； 2025年01月07日竞价强度排名；
开始循环: 0


2025-01-09 01:53:28.761490 - INFO -             总市值     情绪温度  涨停      突破左峰(条件说明)       竞价强度   竞价强度排名  竞价强度排名名次  \
0  2.916942e+09  55.9466  涨停  2025年1月7日的突破左峰  10.122852   3/5395         3   
1  5.488782e+09  81.9364  涨停  2025年1月7日的突破左峰   9.664344  10/5395        10   

   竞价强度排名基数       股票代码 股票市场类型  股票简称  
0      5395  000573.SZ   沪深主板  粤宏远A  
1      5395  002346.SZ   沪深主板  柘中股份  


-------------------当前处理股票-------------------: 000573.SZ
-------------------当前处理股票-------------------: 002346.SZ
------------------循环结束----------------------
******************** 第 47 轮循环 ********************
 当前交易日是: 2025-01-07
 前2个交易日是: 2025-01-03
 前1个交易日是: 2025-01-06
 下1个交易日是: 2025-01-08
 下2个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月03日涨停； 2025年01月06日情绪温度大于50分； 2025年01月06日过左峰并涨停； 2025年01月06日竞价强度排名；
开始循环: 0


2025-01-09 01:53:29.967275 - INFO -             总市值     情绪温度  涨停      突破左峰(条件说明)       竞价强度    竞价强度排名  竞价强度排名名次  \
0  1.051443e+10  84.5829  涨停  2025年1月6日的突破左峰  10.534657    1/5395         1   
1  1.041435e+10  97.3575  涨停  2025年1月6日的突破左峰   8.042269  107/5395       107   
2  2.731903e+09  97.3528  涨停  2025年1月6日的突破左峰   7.969369  129/5395       129   
3  2.436494e+09  93.9176  涨停  2025年1月6日的突破左峰   7.757759  171/5395       171   

   竞价强度排名基数       股票代码 股票市场类型  股票简称  
0      5395  600789.SH   沪深主板  鲁抗医药  
1      5395  600618.SH   沪深主板  氯碱化工  
2      5395  605208.SH   沪深主板   永茂泰  
3      5395  603320.SH   沪深主板  迪贝电气  


-------------------当前处理股票-------------------: 600789.SH
-------------------当前处理股票-------------------: 600618.SH
-------------------当前处理股票-------------------: 605208.SH
-------------------当前处理股票-------------------: 603320.SH
------------------循环结束----------------------
******************** 第 46 轮循环 ********************
 当前交易日是: 2025-01-06
 前2个交易日是: 2025-01-02
 前1个交易日是: 2025-01-03
 下1个交易日是: 2025-01-07
 下2个交易日是: 2025-01-08
 下3个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月02日涨停； 2025年01月03日情绪温度大于50分； 2025年01月03日过左峰并涨停； 2025年01月03日竞价强度排名；
开始循环: 0


...

-------------------当前处理股票-------------------: 600826.SH
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2024-11-15 】 的 开盘价:【 9.21 】大于等于【 2024-11-14 】 的收盘价:【 9.21 】
开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【 2024-11-18 】 的 最高价:【 11.14 】大于等于【 2024-11-15 】 的开盘价:【 9.21 】
2024-11-18 :开盘未涨停：共交易【 28 】次，成功了【 19 】次，胜率【 0.6785714285714286 %】收益率【 0.8886186033000646 】
2024-11-18 :开盘涨停：共交易【 6 】次，成功了【 5 】次，胜率【 0.8333333333333334 %】收益率【 0.18081866604722008 】
2024-11-18 :一字板：共交易【 15 】次，成功了【 15 】次，胜率【 1.0 %】收益率【 1.4567642331612192 】
------------------循环结束----------------------
******************** 第 10 轮循环 ********************
 当前交易日是: 2024-11-14
 前2个交易日是: 2024-11-12
 前1个交易日是: 2024-11-13
 下1个交易日是: 2024-11-15
 下2个交易日是: 2024-11-18
 下3个交易日是: 2024-11-19
沪深主板排除ST，股票市值20-500亿之间； 2024年11月12日涨停； 2024年11月13日情绪温度大于50分； 2024年11月13日过左峰并涨停； 2024年11月13日竞价强度排名；
开始循环: 0


2025-01-09 01:54:18.596842 - INFO -             总市值      情绪温度  涨停        突破左峰(条件说明)      竞价强度    竞价强度排名  竞价强度排名名次  \
0  7.231308e+09   74.4447  涨停  2024年11月13日的突破左峰  7.924091  227/5395       227   
1  2.163717e+09  100.0000  涨停  2024年11月13日的突破左峰  7.354533  415/5395       415   

   竞价强度排名基数       股票代码 股票市场类型  股票简称  
0      5395  002232.SZ   沪深主板  启明信息  
1      5395  000695.SZ   沪深主板  滨海能源  


-------------------当前处理股票-------------------: 002232.SZ
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2024-11-14 】 的 开盘价:【 26.81 】大于等于【 2024-11-13 】 的收盘价:【 26.81 】
开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【 2024-11-15 】 的 最高价:【 29.0 】大于等于【 2024-11-14 】 的开盘价:【 26.81 】
-------------------当前处理股票-------------------: 000695.SZ
2024-11-15 :开盘未涨停：共交易【 29 】次，成功了【 20 】次，胜率【 0.6896551724137931 %】收益率【 0.9703045413828696 】
2024-11-15 :开盘涨停：共交易【 6 】次，成功了【 5 】次，胜率【 0.8333333333333334 %】收益率【 0.18081866604722008 】
2024-11-15 :一字板：共交易【 15 】次，成功了【 15 】次，胜率【 1.0 %】收益率【 1.4567642331612192 】
------------------循环结束----------------------
******************** 第 9 轮循环 ********************
 当前交易日是: 2024-11-13
 前2个交易日是: 2024-11-11
 前1个交易日是: 2024-11-12
 下1个交易日是: 2024-11-14
 下2个交易日是: 2024-11-15
 下3个交易日是: 2024-11-18
沪深主板排除ST，股票市值20-500亿之间； 2024年11月11日涨停； 2024年11月12日情绪温度大于50分； 2024年11月12日过左峰并涨停； 2024年11月12日竞价强度排名；
开始循环: 0
2024-11-14 :开盘未涨停：共交易【 29 】次，成功了【 20 】次，胜率【 0.6896551724137931 %】收益率【 0.9703045413828696 】
2024-11-14 :开盘涨停：共交易【 6 】次，成功了【 5 】

2025-01-09 01:54:26.335789 - INFO -             总市值   情绪温度  涨停       突破左峰(条件说明)      竞价强度   竞价强度排名  竞价强度排名名次  \
0  9.041846e+09  100.0  涨停  2024年11月4日的突破左峰  8.884038  74/5395        74   

   竞价强度排名基数       股票代码 股票市场类型  股票简称  
0      5395  002979.SZ   沪深主板  雷赛智能  


-------------------当前处理股票-------------------: 002979.SZ
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2024-11-05 】 的 开盘价:【 32.0 】大于等于【 2024-11-04 】 的收盘价:【 29.83 】
开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【 2024-11-06 】 的 最高价:【 32.98 】大于等于【 2024-11-05 】 的开盘价:【 32.0 】
2024-11-06 :开盘未涨停：共交易【 30 】次，成功了【 21 】次，胜率【 0.7 %】收益率【 1.0009295413828694 】
2024-11-06 :开盘涨停：共交易【 6 】次，成功了【 5 】次，胜率【 0.8333333333333334 %】收益率【 0.18081866604722008 】
2024-11-06 :一字板：共交易【 15 】次，成功了【 15 】次，胜率【 1.0 %】收益率【 1.4567642331612192 】
------------------循环结束----------------------
******************** 第 2 轮循环 ********************
 当前交易日是: 2024-11-04
 前2个交易日是: 2024-10-31
 前1个交易日是: 2024-11-01
 下1个交易日是: 2024-11-05
 下2个交易日是: 2024-11-06
 下3个交易日是: 2024-11-07
沪深主板排除ST，股票市值20-500亿之间； 2024年10月31日涨停； 2024年11月01日情绪温度大于50分； 2024年11月01日过左峰并涨停； 2024年11月01日竞价强度排名；
开始循环: 0
2024-11-05 :开盘未涨停：共交易【 30 】次，成功了【 21 】次，胜率【 0.7 %】收益率【 1.0009295413828694 】
2024-11-05 :开盘涨停：共交易【 6 】次，成功了【 5 】次，胜率【 0.8333333333333334 %】收益率【 0.18081866604722008 】
2024-11-05 :一字板：共交易【 15 】次，成功了【 1

## 强中选强

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import time
import numpy as np
from tabulate import tabulate
from mindgo_api import *


pd.options.display.max_columns = None  # 显示所有列
pd.options.display.max_rows = None  # 显示所有行
  
part1 = "沪深主板排除ST，股票市值20-500亿之间；"
part2 = "强中选强；"
part3 = "情绪温度80分以上；"
part4 = "竞价强度排名；"


# 获取当前日期和时间
end_date = datetime.now()

tdays_list = get_trade_days(end_date=end_date.strftime('%Y-%m-%d'),count=10)
# 假设要打印过去30天，你可以根据实际需求修改这个天数

for i in range(len(tdays_list) - 1, 0, -1):
    yes_current_date = tdays_list[i-1].strftime('%Y-%m-%d')
    current_date = tdays_list[i].strftime('%Y-%m-%d')
    
    end_date = tdays_list[len(tdays_list)-1].strftime('%Y-%m-%d')
    
    new_query = part1 + " " + yes_current_date + part2 + " " + yes_current_date + part3 + " " + current_date + part4
    print(new_query)
    count = 0
    while count<1:
        print('开始循环:',count)
        time.sleep(1)
        count+=1
        result = query_iwencai(new_query,domain='股票',timeout=10)
        if len(result)>0:
            stock_code_list = result['股票代码'].tolist()
            for code in stock_code_list:
# #                 print(code)
# # #                 JJ(code,current_date)
                value = get_price(securities=[code], end_date= end_date, fre_step = '1d', fields=['open', 'close', 'high', 'low'], skip_paused=True, fq='pre',  bar_count=10,  is_panel=0)
# #                 # 打印收盘价数据
                print(value)
    
            log.info(result)
            break
    
    print('循环结束')

## MZCD金叉

In [58]:
import datetime
import pandas as pd
import time
import numpy as np
from tabulate import tabulate
from mindgo_api import *

def format_date_to_chinese(date_str):
    """
    将格式为%Y-%m-%d的日期字符串转换为中文日期格式（%Y年%m月%d日）
    """
    date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    return date_obj.strftime('%Y年%m月%d日')

pd.options.display.max_columns = None  # 显示所有列
pd.options.display.max_rows = None  # 显示所有行
  
query_3_part1 = "沪深主板排除ST，股票市值20-500亿之间；"
query_3_part2 = "的macd金叉并且KDJ金叉；"
query_3_part3 = "的五日均线大于十日均线；"
query_3_part4 = "的十日均线大于二十日均线；"
query_3_part5 = "的二十日均线大于六十日均线；"
query_3_part6 = "的情绪温度相比前一日提升排名；"
query_3_part7 = "的情绪温度；"
query_3_part8 = "的开盘价大于前一日收盘价的1倍；"


# 日macd金叉并且KDJ处于强势区间***********************
# 最终:开盘未涨停：共交易【 44 】次，成功了【 33 】次，胜率【 0.75 %】收益率【 1.5513345086100703 】
# 最终:开盘涨停：共交易【 5 】次，成功了【 4 】次，胜率【 0.8 %】收益率【 0.2020587764117604 】
# 最终:一字板：共交易【 13 】次，成功了【 12 】次，胜率【 0.9230769230769231 %】收益率【 1.141297141310038 】

# 日macd金叉并且CCI买入信号并且KDJ金叉
# 最终:开盘未涨停：共交易【 42 】次，成功了【 30 】次，胜率【 0.7142857142857143 %】收益率【 1.3178931174019675 】
# 最终:开盘涨停：共交易【 5 】次，成功了【 4 】次，胜率【 0.8 %】收益率【 0.2020587764117604 】
# 最终:一字板：共交易【 12 】次，成功了【 12 】次，胜率【 1.0 %】收益率【 1.203099077274297 】

#  量能放大
# 沪深主板排除ST，股票市值20-500亿之间； 2024-10-29日macd金叉并且KDJ处于强势区间； 2024-10-29日的五日均线大于十日均线； 2024-10-29日的十日均线大于二十日均线； 2024-10-29日的二十日均线大于六十日均线； 2024-10-29情绪温度相比前一日提升排名； 2024-10-29成交额大于前一日成交额的1倍；
# 最终:开盘未涨停：共交易【 44 】次，成功了【 33 】次，胜率【 0.75 %】收益率【 1.5892840858352957 】
# 最终:开盘涨停：共交易【 3 】次，成功了【 3 】次，胜率【 1.0 %】收益率【 0.17172115981904623 】
# 最终:一字板：共交易【 9 】次，成功了【 8 】次，胜率【 0.8888888888888888 %】收益率【 0.7400682572684872 】

# 日macd金叉并且KDJ金叉 ************
# 沪深主板排除ST，股票市值20-500亿之间； 2025-01-06日macd金叉并且KDJ金叉； 2025-01-06日的五日均线大于十日均线； 2025-01-06日的十日均线大于二十日均线； 2025-01-06日的二十日均线大于六十日均线； 2025-01-06情绪温度相比前一日提升排名；
# 最终:开盘未涨停：共交易【 43 】次，成功了【 33 】次，胜率【 0.7674418604651163 %】收益率【 1.6201048251865011 】
# 最终:开盘涨停：共交易【 5 】次，成功了【 4 】次，胜率【 0.8 %】收益率【 0.2020587764117604 】
# 最终:一字板：共交易【 13 】次，成功了【 12 】次，胜率【 0.9230769230769231 %】收益率【 1.141297141310038 】
# 循环500d
# 最终:开盘未涨停：共交易【 356 】次，成功了【 239 】次，胜率【 0.6713483146067416 %】收益率【 6.60705456468123 】
# 最终:开盘涨停：共交易【 12 】次，成功了【 8 】次，胜率【 0.6666666666666666 %】收益率【 0.1415659088022443 】
# 最终:一字板：共交易【 29 】次，成功了【 27 】次，胜率【 0.9310344827586207 %】收益率【 2.491692952827915 】
# 日macd金叉并且KDJ金叉+格局一天
# 最终:开盘未涨停：共交易【 44 】次，成功了【 36 】次，胜率【 0.8181818181818182 %】收益率【 1.7910677350833837 】
# 最终:开盘涨停：共交易【 5 】次，成功了【 4 】次，胜率【 0.8 %】收益率【 0.2020587764117604 】
# 最终:一字板：共交易【 13 】次，成功了【 12 】次，胜率【 0.9230769230769231 %】收益率【 1.141297141310038 】

# 最终：红开+格局一天+日macd金叉并且KDJ金叉
# 最终:开盘未涨停：共交易【 39 】次，成功了【 33 】次，胜率【 0.8461538461538461 %】收益率【 1.9086267682398323 】
# 最终:开盘涨停：共交易【 5 】次，成功了【 4 】次，胜率【 0.8 %】收益率【 0.2020587764117604 】
# 最终:一字板：共交易【 13 】次，成功了【 12 】次，胜率【 0.9230769230769231 %】收益率【 1.141297141310038 】

# 最终:开盘未涨停：共交易【 39 】次，成功了【 33 】次，胜率【 0.8461538461538461 %】收益率【 1.727785283531418 】
# 最终:开盘涨停：共交易【 5 】次，成功了【 4 】次，胜率【 0.8 %】收益率【 0.2020587764117604 】
# 最终:一字板：共交易【 13 】次，成功了【 12 】次，胜率【 0.9230769230769231 %】收益率【 1.141297141310038 】
# 回测 510 天
# 最终:开盘未涨停：共交易【 253 】次，成功了【 186 】次，胜率【 0.7351778656126482 %】收益率【 5.923616301303021 】
# 最终:开盘涨停：共交易【 12 】次，成功了【 8 】次，胜率【 0.6666666666666666 %】收益率【 0.1415659088022443 】
# 最终:一字板：共交易【 31 】次，成功了【 29 】次，胜率【 0.9354838709677419 %】收益率【 2.691463000432095 】

duration = 10
# 获取当前日期和时间
end_date = datetime.date.today()
tdays_list = get_trade_days(end_date=end_date.strftime('%Y-%m-%d'),count=duration)
# 假设要打印过去30天，你可以根据实际需求修改这个天数

buy_1 = 0
sell_1= 0
buy_2 = 0
sell_2= 0
buy_3 = 0
sell_3= 0
rate_1 = 0
rate_2 = 0
rate_3 = 0
for i in range(len(tdays_list)-1, 1, -1):
    current_date = tdays_list[i].strftime('%Y-%m-%d')
    
    end_date = tdays_list[len(tdays_list)-1].strftime('%Y-%m-%d')
    next_date = None
    next_next_date = None
    next_next_next_date = None
    before_before_date = None
    before_date = None
    
    
    print('**'*10,'第',i ,'轮循环','**'*10)
    print(' 当前交易日是:',current_date)
    if i-2 >= 0 and i-2 < len(tdays_list):
        before_before_date = tdays_list[i-2].strftime('%Y-%m-%d')
        print(' 前2个交易日是:',before_before_date)
        
    if i-1 >= 0 and i-1 < len(tdays_list):
        before_date = tdays_list[i-1].strftime('%Y-%m-%d')
        print(' 前1个交易日是:',before_date)
        
    if i+1 < len(tdays_list):
        next_date = tdays_list[i+1].strftime('%Y-%m-%d')
        print(' 下1个交易日是:',next_date)
    
    
    if i+2 < len(tdays_list):
        next_next_date = tdays_list[i+2].strftime('%Y-%m-%d')
        print(' 下2个交易日是:',next_next_date)
    
    
    if i+3 < len(tdays_list):
        next_next_next_date = tdays_list[i+3].strftime('%Y-%m-%d')
        print(' 下3个交易日是:',next_next_next_date)
    
    new_query = ''
    if before_date and current_date:
        new_query = query_3_part1+  \
            " " + format_date_to_chinese(before_date) + query_3_part2+ \
            " " + format_date_to_chinese(before_date) + query_3_part3+ \
            " " + format_date_to_chinese(before_date) + query_3_part4+ \
            " " + format_date_to_chinese(before_date) + query_3_part5+ \
            " " + format_date_to_chinese(before_date) + query_3_part6+ \
            " " + format_date_to_chinese(before_date) + query_3_part7+ \
            " " + format_date_to_chinese(current_date) + query_3_part8
    
    print(new_query)
    count = 0
    while count<1:
        print('开始循环:',count)
        time.sleep(1)
        count+=1
        stock_cnt_1 = 0
        stock_cnt_2 = 0
        stock_cnt_3 = 0
        result = query_iwencai(new_query,domain='股票',timeout=10)
        if len(result)>0:
            result = result.sort_values(by='情绪温度环比增长', ascending=False)
            log.info(result)
            stock_code_list = result['股票代码'].tolist()
            for code in stock_code_list:
                print('-------------------当前处理股票-------------------:',code)
#                 JJ(code,current_date)
                value = get_price(securities=[code], end_date= end_date, fre_step = '1d', fields=['open', 'close', 'high', 'low'], skip_paused=True, fq='pre',  bar_count=duration+100,  is_panel=0)
            
                if before_date and before_date in  value[code].index and current_date and current_date in value[code].index and next_date and next_date in value[code].index  and next_next_date and next_next_date in value[code].index :
#                   
                    buy_flag_1 = value[code].loc[current_date]['open'] < round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_2 = value[code].loc[current_date]['open'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_3 = value[code].loc[current_date]['low'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    sell_flag_1 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['high']
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['open']*1
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['open']
                    sell_flag_1_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['high']
                    if buy_flag_1 and stock_cnt_1==0:
                        print(' 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】大于等于【',before_date,'】 的收盘价:【',value[code].loc[before_date]['close'],'】')
                        buy_1+=1
                        stock_cnt_1+=1
                        if sell_flag_2 or sell_flag_1:
                            print('开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_date]['high']/value[code].loc[current_date]['open']-1
                        elif sell_flag_2_1 or sell_flag_1_1:
                            print(' 格局一天，开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_next_date,'】 的 最高价:【',value[code].loc[next_next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_next_date]['high']/value[code].loc[current_date]['open']-1
                        else:
                            rate_1 += value[code].loc[next_next_date]['close']/value[code].loc[current_date]['open']-1
                        
#                     开盘涨停-非一字板
                    if buy_flag_2 and not buy_flag_3:
                        print('开盘涨停非一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_2+=1
                        stock_cnt_2+=1
                        if sell_flag_1:
                            print('开盘涨停非一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_2+=1
                            rate_2 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_2 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1
                        
                            
#                      开盘涨停一字板
                    if buy_flag_3 :
                        print('一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_3+=1
                        stock_cnt_3+=1
                        if sell_flag_1:
                            print('一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_3+=1
                            rate_3 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_3 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1
                        
            break
    if buy_1>0:
        print(current_date,':开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
    if buy_2>0:
        print(current_date,':开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
    if buy_3>0:
        print(current_date,':一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')
    print('------------------循环结束----------------------')



if buy_1>0:
    print('最终:',current_date,'-开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
if buy_2>0:
    print('最终:',current_date,'-开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
if buy_3>0:
    print('最终:',current_date,'-一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')


******************** 第 9 轮循环 ********************
 当前交易日是: 2025-01-09
 前2个交易日是: 2025-01-07
 前1个交易日是: 2025-01-08
沪深主板排除ST，股票市值20-500亿之间； 2025年01月08日的macd金叉并且KDJ金叉； 2025年01月08日的五日均线大于十日均线； 2025年01月08日的十日均线大于二十日均线； 2025年01月08日的二十日均线大于六十日均线； 2025年01月08日的情绪温度相比前一日提升排名； 2025年01月08日的情绪温度； 2025年01月09日的开盘价大于前一日收盘价的1倍；
开始循环: 0
------------------循环结束----------------------
******************** 第 8 轮循环 ********************
 当前交易日是: 2025-01-08
 前2个交易日是: 2025-01-06
 前1个交易日是: 2025-01-07
 下1个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月07日的macd金叉并且KDJ金叉； 2025年01月07日的五日均线大于十日均线； 2025年01月07日的十日均线大于二十日均线； 2025年01月07日的二十日均线大于六十日均线； 2025年01月07日的情绪温度相比前一日提升排名； 2025年01月07日的情绪温度； 2025年01月08日的开盘价大于前一日收盘价的1倍；
开始循环: 0


2025-01-09 03:01:17.465096 - INFO -         kdj金叉(条件说明)       macd金叉(条件说明)  {(}10日均线{-}20日均线{)}  \
0  2025年01月07日kdj金叉  2025年01月07日macd金叉                0.065   

   {(}20日均线{-}60日均线{)}  {(}5日均线{-}10日均线{)}  {(}开盘价:不复权{/}收盘价:不复权{)}      均线  \
0                0.949               0.349                 1.007625  18.014   

  开盘价:不复权           总市值     情绪温度  情绪温度环比增长 情绪温度环比增长排名  情绪温度环比增长排名名次  \
0    18.5  2.300451e+09  10.7136    1.2416   820/5395           820   

   情绪温度环比增长排名基数  收盘价:不复权       股票代码 股票市场类型  股票简称  
0          5395    18.36  002778.SZ   沪深主板  中晟高科  


-------------------当前处理股票-------------------: 002778.SZ
------------------循环结束----------------------
******************** 第 7 轮循环 ********************
 当前交易日是: 2025-01-07
 前2个交易日是: 2025-01-03
 前1个交易日是: 2025-01-06
 下1个交易日是: 2025-01-08
 下2个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月06日的macd金叉并且KDJ金叉； 2025年01月06日的五日均线大于十日均线； 2025年01月06日的十日均线大于二十日均线； 2025年01月06日的二十日均线大于六十日均线； 2025年01月06日的情绪温度相比前一日提升排名； 2025年01月06日的情绪温度； 2025年01月07日的开盘价大于前一日收盘价的1倍；
开始循环: 0


2025-01-09 03:01:19.074366 - INFO -         kdj金叉(条件说明)       macd金叉(条件说明)  {(}10日均线{-}20日均线{)}  \
0  2025年01月06日kdj金叉  2025年01月06日macd金叉                0.321   
1  2025年01月06日kdj金叉  2025年01月06日macd金叉                0.140   

   {(}20日均线{-}60日均线{)}  {(}5日均线{-}10日均线{)}  {(}开盘价:不复权{/}收盘价:不复权{)}      均线  \
0                0.817               0.027                 1.099952  20.201   
1                0.309               0.012                 1.099448  10.172   

  开盘价:不复权           总市值     情绪温度  情绪温度环比增长 情绪温度环比增长排名  情绪温度环比增长排名名次  \
0   23.11  1.128662e+10  58.7944   55.5141    15/5395            15   
1   11.94  5.488782e+09  57.1744   54.1445    16/5395            16   

   情绪温度环比增长排名基数  收盘价:不复权       股票代码 股票市场类型  股票简称  
0          5395    21.01  603063.SH   沪深主板  禾望电气  
1          5395    10.86  002346.SZ   沪深主板  柘中股份  


-------------------当前处理股票-------------------: 603063.SH
-------------------当前处理股票-------------------: 002346.SZ
------------------循环结束----------------------
******************** 第 6 轮循环 ********************
 当前交易日是: 2025-01-06
 前2个交易日是: 2025-01-02
 前1个交易日是: 2025-01-03
 下1个交易日是: 2025-01-07
 下2个交易日是: 2025-01-08
 下3个交易日是: 2025-01-09
沪深主板排除ST，股票市值20-500亿之间； 2025年01月03日的macd金叉并且KDJ金叉； 2025年01月03日的五日均线大于十日均线； 2025年01月03日的十日均线大于二十日均线； 2025年01月03日的二十日均线大于六十日均线； 2025年01月03日的情绪温度相比前一日提升排名； 2025年01月03日的情绪温度； 2025年01月06日的开盘价大于前一日收盘价的1倍；
开始循环: 0
------------------循环结束----------------------
******************** 第 5 轮循环 ********************
 当前交易日是: 2025-01-03
 前2个交易日是: 2024-12-31
 前1个交易日是: 2025-01-02
 下1个交易日是: 2025-01-06
 下2个交易日是: 2025-01-07
 下3个交易日是: 2025-01-08
沪深主板排除ST，股票市值20-500亿之间； 2025年01月02日的macd金叉并且KDJ金叉； 2025年01月02日的五日均线大于十日均线； 2025年01月02日的十日均线大于二十日均线； 2025年01月02日的二十日均线大于六十日均线； 2025年01月02日的情绪温度相比前一日提升排名； 2025年01月02日的情绪温度； 2025年01月03日的开盘价大于前一日收盘价的1倍；
开始循环: 0
------------------循环结束------------

2025-01-09 03:01:23.893019 - INFO -         kdj金叉(条件说明)       macd金叉(条件说明)  {(}10日均线{-}20日均线{)}  \
0  2024年12月31日kdj金叉  2024年12月31日macd金叉                0.057   

   {(}20日均线{-}60日均线{)}  {(}5日均线{-}10日均线{)}  {(}开盘价:不复权{/}收盘价:不复权{)}     均线  \
0                0.505                0.15                 1.099591  6.796   

  开盘价:不复权           总市值     情绪温度  情绪温度环比增长 情绪温度环比增长排名  情绪温度环比增长排名名次  \
0    8.06  3.184053e+09  83.0698   70.4221     7/5395             7   

   情绪温度环比增长排名基数  收盘价:不复权       股票代码 股票市场类型  股票简称  
0          5395     7.33  603617.SH   沪深主板  君禾股份  


-------------------当前处理股票-------------------: 603617.SH
开盘涨停非一字板：买入买入买入!!!!  该股票在【 2025-01-02 】 的 开盘价:【 8.06 】等于【 2024-12-31 】 的收盘价的1.1倍:【 7.33 】
开盘涨停非一字板：卖出卖出卖出!!!! 该股票在【 2025-01-03 】 的 最高价:【 8.87 】大于等于【 2025-01-02 】 的最高价:【 8.06 】
2025-01-02 :开盘涨停：共交易【 1 】次，成功了【 1 】次，胜率【 1.0 %】收益率【 0.10049627791563265 】
------------------循环结束----------------------
******************** 第 3 轮循环 ********************
 当前交易日是: 2024-12-31
 前2个交易日是: 2024-12-27
 前1个交易日是: 2024-12-30
 下1个交易日是: 2025-01-02
 下2个交易日是: 2025-01-03
 下3个交易日是: 2025-01-06
沪深主板排除ST，股票市值20-500亿之间； 2024年12月30日的macd金叉并且KDJ金叉； 2024年12月30日的五日均线大于十日均线； 2024年12月30日的十日均线大于二十日均线； 2024年12月30日的二十日均线大于六十日均线； 2024年12月30日的情绪温度相比前一日提升排名； 2024年12月30日的情绪温度； 2024年12月31日的开盘价大于前一日收盘价的1倍；
开始循环: 0
2024-12-31 :开盘涨停：共交易【 1 】次，成功了【 1 】次，胜率【 1.0 %】收益率【 0.10049627791563265 】
------------------循环结束----------------------
******************** 第 2 轮循环 ********************
 当前交易日是: 2024-12-30
 前2个交易日是: 2024-12-26
 前1个交易日是: 2024-12-27
 下1个交易日是: 2024-12-31
 下2个交易日是: 2025-

2025-01-09 03:01:28.059690 - INFO -         kdj金叉(条件说明)       macd金叉(条件说明)  {(}10日均线{-}20日均线{)}  \
0  2024年12月27日kdj金叉  2024年12月27日macd金叉                0.057   

   {(}20日均线{-}60日均线{)}  {(}5日均线{-}10日均线{)}  {(}开盘价:不复权{/}收盘价:不复权{)}     均线  \
0                0.136               0.006                 1.004172  7.050   

  开盘价:不复权           总市值   情绪温度  情绪温度环比增长 情绪温度环比增长排名  情绪温度环比增长排名名次  \
0    7.22  2.053095e+10  8.793    4.4847   331/5395           331   

   情绪温度环比增长排名基数  收盘价:不复权       股票代码 股票市场类型  股票简称  
0          5395     7.19  600820.SH   沪深主板  隧道股份  


-------------------当前处理股票-------------------: 600820.SH
 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【 2024-12-30 】 的 开盘价:【 7.22 】大于等于【 2024-12-27 】 的收盘价:【 7.19 】
开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【 2024-12-31 】 的 最高价:【 7.37 】大于等于【 2024-12-30 】 的开盘价:【 7.22 】
2024-12-30 :开盘未涨停：共交易【 1 】次，成功了【 1 】次，胜率【 1.0 %】收益率【 0.020775623268698196 】
2024-12-30 :开盘涨停：共交易【 1 】次，成功了【 1 】次，胜率【 1.0 %】收益率【 0.10049627791563265 】
------------------循环结束----------------------
最终: 2024-12-30 -开盘未涨停：共交易【 1 】次，成功了【 1 】次，胜率【 1.0 %】收益率【 0.020775623268698196 】
最终: 2024-12-30 -开盘涨停：共交易【 1 】次，成功了【 1 】次，胜率【 1.0 %】收益率【 0.10049627791563265 】


## 巨量阴

In [ ]:
import datetime
import pandas as pd
import time
import numpy as np
from tabulate import tabulate
from mindgo_api import *

def format_date_to_chinese(date_str):
    """
    将格式为%Y-%m-%d的日期字符串转换为中文日期格式（%Y年%m月%d日）
    """
    date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    return date_obj.strftime('%Y年%m月%d日')

pd.options.display.max_columns = None  # 显示所有列
pd.options.display.max_rows = None  # 显示所有行
  
part1 = "沪深主板排除ST，股票市值20-500亿之间；"
part2 = "连续涨停非一字板；"
part21 = "缩量涨停；"
part3 = "阴线；"
part4 = "成交量大于前一日成交量的0.9倍；"
part5 = "情绪温度；"
part6 = "情绪温度相比前一日提升排名；"
part7 = "收盘价大于前一日收盘价的1倍；"
part8 = "竞价强度；"
part9 = "上影线较短；"

# 沪深主板排除ST，股票市值20-500亿之间； 
# 2025-01-02号阴线；
# 2025-01-02成交量大于前一日成交量的2倍；
# 2025-01-02情绪温度；
# 2024年12月31号涨停；
# 2025-01-02情绪温度相比前一日提升排名；
# 2025-01-02收盘价大于前一日收盘价的1倍；
# 2025-01-02竞价强度；
# 2025-01-02上影线较短
duration = 200
# 获取当前日期和时间
end_date = datetime.date.today()
tdays_list = get_trade_days(end_date=end_date.strftime('%Y-%m-%d'),count=duration)
# 假设要打印过去30天，你可以根据实际需求修改这个天数

buy_1 = 0
sell_1= 0
buy_2 = 0
sell_2= 0
buy_3 = 0
sell_3= 0
rate_1 = 0
rate_2 = 0
rate_3 = 0
for i in range(len(tdays_list) - 1, 0, -1):
    yes_current_date = tdays_list[i-1].strftime('%Y-%m-%d')
    current_date = tdays_list[i].strftime('%Y-%m-%d')
    
    
    end_date = tdays_list[len(tdays_list)-1].strftime('%Y-%m-%d')
    next_date = None
    next_next_date = None
    
    if i-2 < len(tdays_list):
        before_before_date = tdays_list[i-2].strftime('%Y-%m-%d')
        print(' 前2个交易日是:',before_before_date)
        
    if i-1 < len(tdays_list):
        before_date = tdays_list[i-1].strftime('%Y-%m-%d')
        print(' 前1个交易日是:',before_date)
        
    if i+1 < len(tdays_list):
        next_date = tdays_list[i+1].strftime('%Y-%m-%d')
        print(' 下1个交易日是:',next_date)
    
    
    if i+2 < len(tdays_list):
        next_next_date = tdays_list[i+2].strftime('%Y-%m-%d')
        print(' 下2个交易日是:',next_next_date)
    
    
    if i+3 < len(tdays_list):
        next_next_next_date = tdays_list[i+3].strftime('%Y-%m-%d')
        print(' 下3个交易日是:',next_next_next_date)
        
    new_query = part1+  \
        " " + format_date_to_chinese(before_before_date) + part2+ \
        " " + format_date_to_chinese(before_before_date) + part21+ \
        " " + format_date_to_chinese(before_date)  + part3+ \
        " " + format_date_to_chinese(before_date)  + part4+ \
        " " + format_date_to_chinese(before_date)  + part5+ \
        " " + format_date_to_chinese(before_date)  + part6+ \
        " " + format_date_to_chinese(before_date)  + part8
    
    print(new_query)
    count = 0
    while count<1:
        print('开始循环:',count)
        time.sleep(1)
        count+=1
        stock_cnt_1 = 0
        stock_cnt_2 = 0
        stock_cnt_3 = 0
        result = query_iwencai(new_query,domain='股票',timeout=10)
        if len(result)>0:
            result = result.sort_values(by='情绪温度环比增长', ascending=False)
            stock_code_list = result['股票代码'].tolist()
            for code in stock_code_list:
                print('-------------------当前处理股票-------------------:',code)
#                 JJ(code,current_date)
                value = get_price(securities=[code], end_date= end_date, fre_step = '1d', fields=['open', 'close', 'high', 'low'], skip_paused=True, fq='pre',  bar_count=duration+100,  is_panel=0)
            
                if before_date and before_date in  value[code].index and current_date and current_date in value[code].index and next_date and next_date in value[code].index  and next_next_date and next_next_date in value[code].index :
#                   
                    buy_flag_1 = value[code].loc[current_date]['open'] < round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_2 = value[code].loc[current_date]['open'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    buy_flag_3 = value[code].loc[current_date]['low'] >= round(value[code].loc[before_date]['close']*1.1,2)
                    
                    sell_flag_1 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['high']
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2 = value[code].loc[next_date]['high'] >= value[code].loc[current_date]['open']*1
#                     sell_flag_2 = value[code].loc[next_next_date]['high'] >= value[code].loc[next_date]['close']
                    sell_flag_2_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['open']
                    sell_flag_1_1 = value[code].loc[next_next_date]['high'] >= value[code].loc[current_date]['high']
                    if buy_flag_1 and stock_cnt_1==0:
                        print(' 开盘未涨停，只买一只：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】大于等于【',before_date,'】 的收盘价:【',value[code].loc[before_date]['close'],'】')
                        buy_1+=1
                        stock_cnt_1+=1
                        if sell_flag_2 or sell_flag_1:
                            print('开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_date]['high']/value[code].loc[current_date]['open']-1
                        elif sell_flag_2_1 or sell_flag_1_1:
                            print(' 格局一天，开盘未涨停，只买一只：卖出卖出卖出!!!! 该股票在【',next_next_date,'】 的 最高价:【',value[code].loc[next_next_date]['high'],'】大于等于【',current_date,'】 的开盘价:【',value[code].loc[current_date]['open'],'】')
                            sell_1+=1
                            rate_1 += value[code].loc[next_next_date]['high']/value[code].loc[current_date]['open']-1
                        else:
                            rate_1 += value[code].loc[next_next_date]['close']/value[code].loc[current_date]['open']-1
                        
#                     开盘涨停-非一字板
                    if buy_flag_2 and not buy_flag_3:
                        print('开盘涨停非一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_2+=1
                        stock_cnt_2+=1
                        if sell_flag_1:
                            print('开盘涨停非一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_2+=1
                            rate_2 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_2 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1
                        
                            
#                      开盘涨停一字板
                    if buy_flag_3 :
                        print('一字板：买入买入买入!!!!  该股票在【',current_date,'】 的 开盘价:【',value[code].loc[current_date]['open'],'】等于【',before_date,'】 的收盘价的1.1倍:【',value[code].loc[before_date]['close'],'】')
                        buy_3+=1
                        stock_cnt_3+=1
                        if sell_flag_1:
                            print('一字板：卖出卖出卖出!!!! 该股票在【',next_date,'】 的 最高价:【',value[code].loc[next_date]['high'],'】大于等于【',current_date,'】 的最高价:【',value[code].loc[current_date]['high'],'】')
                            sell_3+=1
                            rate_3 += value[code].loc[next_date]['high']/value[code].loc[current_date]['high']-1
                        else:
                            rate_3 += value[code].loc[next_date]['close']/value[code].loc[current_date]['high']-1
                        
            log.info(result)
            break
    if buy_1>0:
        print(current_date,':开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
    if buy_2>0:
        print(current_date,':开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
    if buy_3>0:
        print(current_date,':一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')
    print('------------------循环结束----------------------')


if buy_1>0:
    print('最终:',current_date,'-开盘未涨停：共交易【',buy_1,'】次，成功了【',sell_1,'】次，胜率【',sell_1/buy_1,'%】收益率【',rate_1,'】')
if buy_2>0:
    print('最终:',current_date,'-开盘涨停：共交易【',buy_2,'】次，成功了【',sell_2,'】次，胜率【',sell_2/buy_2,'%】收益率【',rate_2,'】')
if buy_3>0:
    print('最终:',current_date,'-一字板：共交易【',buy_3,'】次，成功了【',sell_3,'】次，胜率【',sell_3/buy_3,'%】收益率【',rate_3,'】')
